In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
545,2024-07-24,Porto Rico Bsn,21:00,Osos de Manati,Capitanes De Arecibo,1.49,2.50,195.5,1.82,1.88,0.0,0.0,0.0,04Aeyn0n,0.671141,0.400000,0.549451,0.531915,0.071141,219.368,58.289578,0.265716,1.2,1.643168,1.369306,178.45,2.118,0.496155,0.234256,27.0,172.904,18.818521,0.108838,1.8,1.643168,0.912871,160.05,1.792,0.390730,0.218041,-27.0,83,97,2.15,1.65,166.488,156.102,0.357984,0.022933,NaN,2.89,0.578,0.847751,0.639262,0.7,0.060738,-5.00,-1.000,-1.500000,0.545809,0.2,-0.345809
546,2024-07-24,Venezuela Superliga,20:00,Cocodrilos,Broncos,1.36,2.97,169.5,1.84,1.84,0.0,0.0,0.0,fyg2WODN,0.735294,0.336700,0.543478,0.543478,0.071994,116.904,17.768661,0.151994,2.4,1.341641,0.559017,130.29,1.340,0.174213,0.130009,42.0,186.820,76.941445,0.411848,1.2,1.643168,1.369306,129.22,2.290,1.169444,0.510674,-48.0,101,91,1.29,1.42,103.134,189.428,0.525839,0.000000,NaN,-0.10,-0.020,-18.000000,0.000000,0.0,0.000000,-2.19,-0.438,-4.497717,0.518482,0.5,-0.018482
547,2024-07-24,Porto Rico Bsn,21:00,Mets de Guaynabo,Leones De Ponce,1.57,2.31,172.5,1.80,1.88,0.0,0.0,0.0,l0VpPKEL,0.636943,0.432900,0.555556,0.531915,0.069843,145.236,27.884871,0.191997,1.8,1.643168,0.912871,188.34,1.694,0.363772,0.214741,-3.0,202.616,51.347681,0.253424,1.8,1.643168,0.912871,148.33,2.314,0.652097,0.281805,10.0,86,91,2.19,1.63,153.106,244.626,0.269721,0.030744,NaN,2.95,0.590,0.966102,0.667902,0.8,0.132098,-2.28,-0.456,-2.872807,0.330666,0.3,-0.030666
548,2024-07-24,Venezuela Superliga,20:00,Guaiqueries,Spartans,1.44,2.65,154.5,1.81,1.87,0.0,0.0,0.0,Qe1oz7Eb,0.694444,0.377358,0.552486,0.534759,0.071803,156.438,63.497562,0.405896,3.0,0.000000,0.000000,88.32,1.998,0.744493,0.372619,45.0,124.112,22.212563,0.178972,2.4,1.341641,0.559017,107.67,1.480,0.525309,0.354939,65.0,69,97,1.28,1.11,139.842,165.218,0.418386,0.023058,NaN,2.00,0.400,1.100000,0.633874,0.8,0.166126,1.20,0.240,6.875000,0.000000,0.0,0.000000
549,2024-07-24,República Dominicana Lnb,21:00,Soles,Indios,1.76,1.97,168.5,1.80,1.90,0.0,0.0,0.0,lblvQKw3,0.568182,0.507614,0.555556,0.526316,0.075796,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,77,2.65,1.97,0.000,0.000,0.079621,0.038222,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
550,2024-07-24,Nova Zelândia Nbl,05:00,Auckland Tuatara,Franklin Bulls,1.34,3.08,178.5,1.87,1.79,0.0,0.0,0.0,OCWaqtfC,0.746269,0.324675,0.534759,0.558659,0.070944,145.566,52.822860,0.362879,3.0,0.000000,0.000000,230.51,1.496,0.641272,0.428658,96.0,186.432,105.702494,0.566976,1.8,1.643168,0.912871,266.24,1.870,0.818810,0.437866,43.0,89,104,2.59,2.56,129.936,184.860,0.556727,0.030912,NaN,1.05,0.210,1.619048,0.000000,0.0,0.000000,2.92,0.584,3.561644,0.000000,0.0,0.000000
551,2024-07-24,Colômbia Lbp,19:40,Titanes de Barranquilla,Toros del Valle,1.18,4.10,164.5,1.83,1.83,0.0,0.0,0.0,na5YfLBk,0.847458,0.243902,0.546448,0.546448,0.091360,114.600,22.161241,0.193379,1.8,1.643168,0.912871,84.46,1.392,0.204010,0.146559,12.0,342.670,210.512320,0.614330,1.2,1.643168,1.369306,716.25,4.212,3.008067,0.714166,1.0,82,75,1.03,9.55,110.624,326.886,0.782103,0.000000,NaN,0.24,0.048,3.750000,0.778421,0.7,-0.078421,0.28,0.056,55.357143,0.297135,0.3,0.002865
552,2024-07-24,Mu

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
